1. 주어진 바닐라 데이터를 가지고 `이미지와 해당하는 클래스 Label` (18개의 클래스 중 하나)을 생성할 수 있는 `Pytorch Dataset Class`를 직접 생성해보세요.<br>
만약 이런 과정을 직접 만들어보는게 처음이시라면 생각보다 시간이 오래 걸릴 것이라 예상합니다. 하지만 이 과정은 캠퍼분들 스스로 반드시 해보길 권장합니다. <br>
18개의 클래스를 만드셨다면, 그 타겟 클래스의 분포도 다시 한번 확인해보면 좋겠습니다.

<br><br>

2. 강의때 보여드렸던 torchvision에 내장된 여러 Augmentation 함수와 albumentation 라이브러리의 여러 transform 기법을 적용해보세요. <br>적용해 보신 뒤에 실제로 어떻게 변환되어 나오는지 확인해보세요. <br>아마 plot형태로 그려서 확인해야 할거에요. <br>그리고 이러한 Transforms를  추가한 Dataset이 과연 어느 정도의 성능을 가지는지 체크해보세요. <br>혹여 너무 무거운 프로세스라면 생각보다 느리게 동작하겠죠? 

In [ ]:
import os
from enum import Enum
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image

In [ ]:
PATH = 'my/train/train'
CSV_PATH = os.path.join(PATH, 'train.csv')
IMAGE_PATH = os.path.join(PATH, 'images')

# Augmentation class
- torch vision을 사용해 Augmentation 

In [ ]:
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop, ColorJitter, RandomRotation

class CustonAugmentation:
    """
    Augmentation을 위한 trainsformer를 정의 
    여기선 어떻게 바꿀지만 정의 한다.
    실제 Augmentation이 진행되는 곳은?
    
    """
    def __init__(self, train = True, resize = [512, 384]):
        self.triansformation = {}
        if train:
            self.transformer['train'] = Compose([
                CenterCrop((320, 256)),
                Resize(resize, Image.BILINEAR), # resize 크기로 줄이거나 늘릴건데 보간법은 Image.BILINEAR
                RandomRotation([-10,10])
            ])
            
        else:
            self.transformer['val'] = Compose([
                CenterCrop((320, 256)),
                Resize(resize, Image.BILINEAR),
                RandomRotation([-10,10])
            ])
    
    

# Label Calss
- 그냥 ipynb라면 일반 변수로 둘 수 있지만
- .py 로 변경을 생각하고 만들었기 때문에 class화 한 것이다.

In [ ]:

class MaskLabels(int, Enum):
    MASK = 0
    INCORRECT = 1
    NORMAL = 2
    
class GenderLabels(int, Enum):
    MALE = 0
    FEMALE = 1
    
    # 객체 생성 없이 class명으로 바로 접근하기 위한 classmethod 선언
    @classmethod
    def from_str(cls, value):
        value = value.lower()
        if value == 'male':
            return cls.MALE
        elif value == 'female':
            return cls.FEMALE
        else:
            raise ValueError(f"value must be 'male' or 'female', not {value}")
            
class AgeLabels(int, Enum):
    # 클래스 변수
    YOUNG = 0
    MID = 1
    OLD = 2
    
    @classmethod
    def from_int(cls, value):
        try:
            value = int(value)
        except:
            raise ValueError(f"value type must be 'int' not {value}:{type(value)}")
        
        if value<30:
            return cls.YOUNG
        elif value>=30 and value<60:
            return cls.MID
        else:
            return cls.OLD
        

# Mask Base Dataset
- init 을 위해 setup(path를 읽고, labeling) 함수 추가
- 포인트는 pandas를 쓰지 않고 접근해본 점

In [ ]:
class MaskBaseDataset(Dataset):
    num_classse = 3*2*3
    
    _file_name = {
        "mask1":MaskLabels.MASK,
        "mask2":MaskLabels.MASK,
        "mask3":MaskLabels.MASK,
        "mask4":MaskLabels.MASK,
        "mask5":MaskLabels.MASK,
        "incorrect_maks":MaskLabels.INCORRECT,
        "normal":MaskLabels.NORMAL
    }  
    image_paths=[]
    mask_labels=[]
    gender_labels=[]
    age_labels=[]
    
    def __init__(self, image_dir, transform = None):
        super(CustomDataset,self).__init__()
        
        self.img_dir = img_dir
        self.setup()
    
        self.transform = transform
    
    # class 변수 setup
    def setup(self):
        # image_path / mask / gender / age
        image_dir_list = os.listdir(self.img_dir)
        
        for image_dir_name in image_dir_list:
            if image_dir_name.startswith('.'): # . 시작 폴더 제외
                continue
                
            id_, gender, _, age = folder_name.split('_') 
            age = AgeLabels.from_int(age)
            gender = GenderLabels.from_str(gender)
            
            image_files = os.listdir(image_dir_name)
            
            for file_name in image_files:
                
                img_name ,ext = os.path.splitext(file_name)
                
                if img_name not in _file_name: # . 시작 파일 제외
                    continue
                
                img_path = os.path.join(self.img_dir, image_dir_name, file_name)
                mask = _filename(img_name)
                
                self.image_paths.append(img_path)
                self.mask_labels.append(mask)
                self.age_labels.append(age)
                self.gender_labels.append(gender)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        mask = self.mask_labels[index]
        age = self.age_labels[index]
        gender = self.gender_laels[index]
        
        label = mask*6 + age*3 + gender*1
        
        # transformation을 통한 Augmentation
        image_transform = self.transform(image = np.array(image))['image']
        return image_transform, label
        